In [167]:
from pymongo import MongoClient
import time
import datetime
from pprint import pprint
import re

In [168]:
def get_date(fmt, ts=None):
    ts = int(time.time()) if ts is None else int(ts)
    date = datetime.datetime.fromtimestamp(ts).strftime(fmt)
    return date

def parse_date(fmt, date):
    ts = time.mktime(datetime.datetime.strptime(date, fmt).timetuple())
    return ts

In [214]:
class Agent(object):
    def __init__(self, data):
        self.data = data
        
    def name(self):
        return self.data['Name']
    
    def __str__(self):
        return self.name()

class Price(object):
    def __init__(self, itinerary, data):
        self.data = data
        self.itinerary = itinerary
        
    def price(self):
        return self.data['Price']
    
    def agent(self):
        return [self.itinerary.flight.agent(id).name() for id in self.data['Agents']]
    
    def __str__(self):
        return "{}: {}€".format(self.agent(), self.price())

class Itinerary(object):
    def __init__(self, flight, data):
        self.data = data
        self.flight = flight
        
    def cheapest(self):
        return Price(self, min(self.data['PricingOptions'], key=lambda opts: opts['Price']))
        
    def most_expensive(self):
        return Price(self, max(self.data['PricingOptions'], key=lambda opts: opts['Price']))

class Flight(object):
    def __init__(self, data):
        self.data = data
        self.itineraries = [Itinerary(self, itinerary) for itinerary in self.data['Itineraries']]
        self.agents = {data['Id']: Agent(data) for data in self.data['Agents']}
        
    def cheapest(self):
        return min(self.itineraries, key=lambda it: it.cheapest().price()).cheapest()
    
    def most_expensive(self):
        return max(self.itineraries, key=lambda it: it.most_expensive().price()).most_expensive()
    
    def agent(self, id):
        return self.agents[id]
    
    def date(self):
        return self.data['_id']['date']
    
    def departure(self):
        return self.data['_id']['departure']
    
    def date_timestamp(self):
        return parse_date('%Y-%m-%d', self.data['_id']['date'])
    
    def __str__(self):
        return "{{{}->{}}} {} - {}".format(self.date(), self.departure(), self.cheapest(), self.most_expensive())

In [215]:
class SkyscannerAPI(object):
    def __init__(self):
        self.client = MongoClient()
        self.db = self.client.skyscanner
        self.itineraries = self.db.itineraries
        self.carriers = self.db.carriers
        
    def find_flight(self, departure, origin, destination):
        cursor = self.itineraries.find({
            '_id.departure': departure,
            '_id.origin': origin,
            '_id.destination': destination
        })
        
        sorted_flights = sorted([Flight(data) for data in cursor], key=lambda f: f.date_timestamp())
        unique_flights = {f.date(): f for f in sorted_flights}
        return list(unique_flights.values())

In [236]:
api = SkyscannerAPI()
flights = api.find_flight('2017-06-30', 'BCN', 'LHR')

for f in flights:
    print(f)

{2017-04-01->2017-06-30} ['Iberia']: 88.92€ - ['Austrian Airlines']: 3005.43€
{2017-04-02->2017-06-30} ['eurowings']: 119.99€ - ['Alitalia']: 622.5€
{2017-04-03->2017-06-30} ['Tix.es']: 100.47€ - ['Austrian Airlines']: 3005.43€
{2017-04-04->2017-06-30} ['Travelgenio']: 85.15€ - ['Austrian Airlines']: 3005.43€
{2017-04-05->2017-06-30} ['Viajes El Corte Ingles']: 106.92€ - ['Austrian Airlines']: 3005.43€
{2017-04-06->2017-06-30} ['eDreams']: 94.37€ - ['Austrian Airlines']: 3005.43€
{2017-04-07->2017-06-30} ['Travelgenio']: 85.02€ - ['Austrian Airlines']: 3005.43€
{2017-04-08->2017-06-30} ['Iberia']: 88.92€ - ['Austrian Airlines']: 3005.43€
{2017-04-09->2017-06-30} ['Travelgenio']: 85.06€ - ['Austrian Airlines']: 3005.43€
{2017-04-13->2017-06-30} ['Travelgenio']: 85.42€ - ['lol.travel']: 621.94€
{2017-04-14->2017-06-30} ['Tripair']: 87.92€ - ['Austrian Airlines']: 3005.43€
{2017-04-15->2017-06-30} ['Tripair']: 87.92€ - ['Austrian Airlines']: 3005.43€
{2017-04-16->2017-06-30} ['Tripair']: 